In [33]:
import pickle
import collections
from itertools import combinations
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [10]:
with open('./data/project_authors_2016_q1_k25core.pickle', 'rb') as picklefile:
    project_authors_q1 = pickle.load(picklefile)

In [ ]:
with open('./data/project_authors_2016_q2_k25core.pickle', 'rb') as picklefile:
    project_authors_q2 = pickle.load(picklefile)

In [21]:
commit_index = collections.defaultdict(dict)
for _, row in tqdm_notebook(project_authors_q1.iterrows()):
    commit_index[row['project_id']][row['author_id']]=row['commit_times']

In [26]:
combinations(commit_index[49061893],2).next()

(35842, 6611460)

In [38]:
def construct_cci(commits):
    authors = commits['author_id'].unique()
    projects = commits['project_id'].unique()
    print('Constructing weighted graph for commits with {} authors and {} projects'.format(len(authors), len(projects)))
    weights = collections.defaultdict(list)
    neighbours = collections.defaultdict(set)
    commit_index = collections.defaultdict(dict)
    for _, row in tqdm_notebook(project_authors_q1.iterrows()):
        commit_index[row['project_id']][row['author_id']]=row['commit_times']
    for project, by_author in tqdm_notebook(commit_index.iteritems(), total=len(projects)):
        for a, b in combinations(by_author, 2):
            if a > b:
                a, b = b, a
            weights[(a,b)].append(min(by_author[a], by_author[b]))
    final_weights=[(edge, np.log(sum(weights[edge]))) for edge in weights]
    for edge, weight in tqdm_notebook(final_weights):
        if weight > 0:
            a, b = edge
            neighbours[a].add(b)
            neighbours[b].add(a)
    return neighbours, final_weights

In [39]:
nbrs, weights = construct_cci(project_authors_q1)

Constructing weighted graph for commits with 3053 authors and 2162 projects


In [43]:
with open('./data/2016_q1_cci-weights_kcore.pickle', 'wb') as picklefile:
    pickle.dump(weights, picklefile)
with open('./data/2016_q1_cci-neighbours_kcore.pickle', 'wb') as picklefile:
    pickle.dump(nbrs, picklefile)

In [44]:
nbrs, weights = construct_cci(project_authors_q2)

Constructing weighted graph for commits with 0 authors and 0 projects


In [45]:
with open('./data/2016_q2_cci-weights_kcore.pickle', 'wb') as picklefile:
    pickle.dump(weights, picklefile)
with open('./data/2016_q2_cci-neighbours_kcore.pickle', 'wb') as picklefile:
    pickle.dump(nbrs, picklefile)